# Accessing SQLite Databases Using Python and Pandas

## Python and SQL

When you open a csv in python and assign it a variable name, you are using your computers memory to save that variable.

Accessing data from a database like SQL is not only more efficient, but also it allows you to subset and import only the parts of the data that you need.

### The `sqlite3` module

The [sqlite3](https://docs.python.org/3/library/sqlite3.html) module provides a straightforward interfafce for interacting with SQLite databases. 

A connection object is created using `sqlite3.connect()`; the connection must be closed at the end of the session with the `.close()` command. 

While the connection is open, any interactions with the database require you to make a cursor object with the `.cursor()` command.

The cursor is then ready to perform all kinds of operations with `.execute()`.

In [4]:
import sqlite3
import os
from sys import platform

# set working directory

print(os.getcwd())

if platform == 'linux':
    os.chdir('/home/meelyn/Documents/cloud_development/sw-python-ecology-lesson/')
else:
    os.chdir('/home/meelyn_pandit/Documents/cloud_development/tutorials/software-carpentry/sw-python-ecology-lesson/')

print(platform)

# Createa SQL connection to our SQLite database
con = sqlite3.connect('data/portal_mammals.sqlite')

cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute('select * from species;'):
    print(row)

# Be sure to close the connection
con.close()

/home/meelyn/Documents/cloud_development/sw-python-ecology-lesson
linux
('AB', 'Amphispiza', 'bilineata', 'Bird')
('AH', 'Ammospermophilus', 'harrisi', 'Rodent')
('AS', 'Ammodramus', 'savannarum', 'Bird')
('BA', 'Baiomys', 'taylori', 'Rodent')
('CB', 'Campylorhynchus', 'brunneicapillus', 'Bird')
('CM', 'Calamospiza', 'melanocorys', 'Bird')
('CQ', 'Callipepla', 'squamata', 'Bird')
('CS', 'Crotalus', 'scutalatus', 'Reptile')
('CT', 'Cnemidophorus', 'tigris', 'Reptile')
('CU', 'Cnemidophorus', 'uniparens', 'Reptile')
('CV', 'Crotalus', 'viridis', 'Reptile')
('DM', 'Dipodomys', 'merriami', 'Rodent')
('DO', 'Dipodomys', 'ordii', 'Rodent')
('DS', 'Dipodomys', 'spectabilis', 'Rodent')
('DX', 'Dipodomys', 'sp.', 'Rodent')
('EO', 'Eumeces', 'obsoletus', 'Reptile')
('GS', 'Gambelia', 'silus', 'Reptile')
('NL', 'Neotoma', 'albigula', 'Rodent')
('NX', 'Neotoma', 'sp.', 'Rodent')
('OL', 'Onychomys', 'leucogaster', 'Rodent')
('OT', 'Onychomys', 'torridus', 'Rodent')
('OX', 'Onychomys', 'sp.', 'Roden

## Queries

One of the most common ways to interact with a database is by querying: retrieving data based on some search parameters.

Use a SELECT statement string.

The query is returned as a single tuple or a tuple of tuples. 

Add a WHERE statement to filter your results based on some parameter


In [7]:
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect('data/portal_mammals.sqlite')

cur = con.cursor()

# Return all results of query
cur.execute('select plot_id from plots where plot_type="Control"')
print(cur.fetchall())

# Return first result of query
cur.execute('select species from species where taxa="Bird"')
print(cur.fetchone())

# Be sure to close connection
con.close()

[(2,), (4,), (8,), (11,), (12,), (14,), (17,), (22,)]
('bilineata',)


## Accessing data stored in SQLite using Python and Pandas

Using pandas, we can import results of an SQLite query into a dataframe.

Note that you can use the same SQL commands/syntax that we used in the SQLite lesson.

In [11]:
import pandas as pd
import sqlite3

# Read sqlite query results into a pandas Dataframe
con = sqlite3.connect('data/portal_mammals.sqlite')
df = pd.read_sql_query('select * from surveys', con)
# print(df)

# Verify that results of SQL query is stored in the dataframe
print(df.head())

con.close()

   record_id  month  day  year  plot_id species_id sex  hindfoot_length  \
0          1      7   16  1977        2         NL   M             32.0   
1          2      7   16  1977        3         NL   M             33.0   
2          3      7   16  1977        2         DM   F             37.0   
3          4      7   16  1977        7         DM   M             36.0   
4          5      7   16  1977        3         DM   M             35.0   

   weight  
0     NaN  
1     NaN  
2     NaN  
3     NaN  
4     NaN  


## Storing data: CSV vs SQLite

Storing your data in an SQLite database can provide substantial performance improvements when reading/writing compared to CSV.

The difference in performance becomes more noticeable as the size of the dataset grows.

### Challenge - SQL

1. Create a query that contains survey data collected between 1998-2001 for observations of sex 'male' or 'female' that includes observation's genus and species and site type for the sample. How many records are returned?

2. Create a dataframe that contains the total number of observations (count) made for all years, and sum of observation weights for each site, ordered by site ID.

In [28]:
# 1.
import sqlite3
import pandas as pd

# Read sqlite queries
con = sqlite3.connect('data/portal_mammals.sqlite')

cur = con.cursor() # what does this do??? used to fetch data and create tables?

# Return all results of query: year, plot type (site type), genus, species, and sex
# from the join of the tables surveys, plots, and species, for the years 1998-2001 where sex is 'M' or 'F'.

cur.execute('select surveys.year, surveys.plot_id, plots.plot_type, species.genus, species.species, surveys.sex \
            from surveys inner join plots on surveys.plot_id = plots.plot_id inner join species on\
            surveys.species_id = species.species_id where surveys.year>=1998 and surveys.year<=2001 \
            and ( surveys.sex = "M" OR surveys.sex="F")')

print('The query returned ' + str(len(cur.fetchall())) + ' records.')


# 2.
# Create two sqlite queries results, read as pandas DataFrame
# Include 'year' in both queries so we have something to merge (join) on.
df1 = pd.read_sql_query('select year, count(*) from surveys group by year', con)
df2 = pd.read_sql_query('select year, plot_id, sum(weight) from surveys group by \
                        year, plot_id order by plot_id asc', con)

# Turn the plot_id column values into column names by pivoting
df2 = df2.pivot(index='year', columns='plot_id')['sum(weight)']
df = pd.merge(df1, df2, on='year')

# Verify that result of the SQL queries is stored in the combined dataframe
print(df.head())

con.close()

The query returned 5546 records.
   year  count(*)       1       2       3       4       5       6      7  \
0  1977       503   567.0   784.0   237.0   849.0   943.0   578.0  202.0   
1  1978      1048  4628.0  4789.0  1131.0  4291.0  4051.0  2371.0   43.0   
2  1979       719  1909.0  2501.0   430.0  2438.0  1798.0   988.0  141.0   
3  1980      1415  5374.0  4643.0  1817.0  7466.0  2743.0  3219.0  362.0   
4  1981      1472  6229.0  6282.0  1343.0  4553.0  3596.0  5430.0   24.0   

        8  ...      15     16      17      18     19      20     21      22  \
0   595.0  ...    48.0  132.0  1102.0   646.0  336.0   640.0   40.0   316.0   
1  3669.0  ...   734.0  548.0  4971.0  4393.0  124.0  2623.0  239.0  2833.0   
2  1954.0  ...   472.0  308.0  3736.0  3099.0  379.0  2617.0  157.0  2250.0   
3  3596.0  ...  1071.0  529.0  5877.0  5075.0  691.0  5523.0  321.0  3763.0   
4  4946.0  ...  1083.0  176.0  5050.0  4773.0  410.0  5379.0  600.0  5268.0   

      23      24  
0  169.0     NaN

## Storing data: Create new tables using Pandas

We can also use pandas to create new tables within an SQLite database. Here, we re-do an exercise we did beffore with CSV files using our SQLite database.

We first read in our survey data, then select only those survey results for 2002, and then save it out to its own table so we can work with it on its own later.

In [31]:
import pandas as pd
import sqlite3

con = sqlite3.connect('data/portal_mammals.sqlite')

# Load the data into a dataframe
surveys_df = pd.read_sql_query('select * from surveys', con)

# Select only data for 2002
surveys2002 = surveys_df[surveys_df.year == 2002]
print(surveys2002.head())

# Write the new Dataframe to a new SQLite table
surveys2002.to_sql('surveys2002', con, if_exists='replace')

print(surveys2002.head())
con.close()

       record_id  month  day  year  plot_id species_id   sex  hindfoot_length  \
33320      33321      1   12  2002        1         DM     M             38.0   
33321      33322      1   12  2002        1         DO     M             37.0   
33322      33323      1   12  2002        1         PB     M             28.0   
33323      33324      1   12  2002        1         AB  None              NaN   
33324      33325      1   12  2002        1         DO     M             35.0   

       weight  
33320    44.0  
33321    58.0  
33322    45.0  
33323     NaN  
33324    29.0  
       record_id  month  day  year  plot_id species_id   sex  hindfoot_length  \
33320      33321      1   12  2002        1         DM     M             38.0   
33321      33322      1   12  2002        1         DO     M             37.0   
33322      33323      1   12  2002        1         PB     M             28.0   
33323      33324      1   12  2002        1         AB  None              NaN   
33324      3

## Saving changes to sqlite database

In [40]:
# connect to database
con = sqlite3.connect('data/portal_mammals.sqlite')

# read results into dataframe
df1 = pd.read_sql_query('select surveys.year, plots.plot_type, species.genus, species.species_id, surveys.sex \
                        from surveys inner join plots on surveys.plot_id = plots.plot_id inner join species on \
                        surveys.species_id = species.species_id where surveys.year >=1998 and surveys.year <=2001 \
                        and ( surveys.sex = "M" or surveys.sex = "F")', con)

df1.to_sql('New Table 1', con, if_exists='replace')

# We already have the 'df' dataframe created in the earlier exercise
df.to_sql('new_table_2', con, if_exists='replace')
cur = con.cursor()

cur.execute('select * from new_table_2')
print(cur.fetchall())

# Close the connection
con.close()


[(0, 1977, 503, 567.0, 784.0, 237.0, 849.0, 943.0, 578.0, 202.0, 595.0, 894.0, None, 1217.0, 741.0, 571.0, 802.0, 48.0, 132.0, 1102.0, 646.0, 336.0, 640.0, 40.0, 316.0, 169.0, None), (1, 1978, 1048, 4628.0, 4789.0, 1131.0, 4291.0, 4051.0, 2371.0, 43.0, 3669.0, 5068.0, None, 3857.0, 4036.0, 1651.0, 2897.0, 734.0, 548.0, 4971.0, 4393.0, 124.0, 2623.0, 239.0, 2833.0, None, None), (2, 1979, 719, 1909.0, 2501.0, 430.0, 2438.0, 1798.0, 988.0, 141.0, 1954.0, 3842.0, 112.0, 1819.0, 3423.0, 1851.0, 3181.0, 472.0, 308.0, 3736.0, 3099.0, 379.0, 2617.0, 157.0, 2250.0, 137.0, 901.0), (3, 1980, 1415, 5374.0, 4643.0, 1817.0, 7466.0, 2743.0, 3219.0, 362.0, 3596.0, 6054.0, 548.0, 3717.0, 6290.0, 4534.0, 3898.0, 1071.0, 529.0, 5877.0, 5075.0, 691.0, 5523.0, 321.0, 3763.0, 742.0, 4392.0), (4, 1981, 1472, 6229.0, 6282.0, 1343.0, 4553.0, 3596.0, 5430.0, 24.0, 4946.0, 6812.0, 449.0, 4988.0, 6880.0, 6339.0, 4762.0, 1083.0, 176.0, 5050.0, 4773.0, 410.0, 5379.0, 600.0, 5268.0, 57.0, 3987.0), (5, 1982, 1978, 66

## Key Points

* sqlite3 provides a SQL-like interface to read, query, and write SQL databases from Python.
* sqlite3 can be used with Pandas to read SQL data to the familiar Pandas dataframe
* Pandas and sqlite3 can also be used to transfer between csv and sql formats